In [1]:
import tensorflow as tf

if tf.test.gpu_device_name(): 
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Please install GPU version of TF


In [204]:
import fiftyone as fo
import pandas as pd
from fiftyone import ViewField as F

my_classes = ["Frog", "Fish", "Bird"]
export_dir = "/kaggle/cv/p6"

dataset = fo.zoo.load_zoo_dataset(
    "open-images-v7",
    split="train",
    label_types=["detections"],
    classes=my_classes,
    max_samples=1000,
    shuffle=True
)

dataset = dataset.filter_labels("ground_truth", F("label").is_in(my_classes))

Necessary images already downloaded
Existing download of split 'train' is sufficient
Loading existing dataset 'open-images-v7-train-1000'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [205]:
# Calculate the number of samples to include in the training set
num_samples = len(dataset)
train_samples = int(num_samples * 0.7)

# Split the dataset
train_dataset = dataset.take(train_samples)
test_dataset = dataset.skip(train_samples)

#view the datasets to verify the split
print(f"Training dataset size: {len(train_dataset)}")
print(f"Testing dataset size: {len(test_dataset)}")

Training dataset size: 700
Testing dataset size: 300


In [206]:
train_dir = "/kaggle/cv/train"
test_dir = "/kaggle/cv/test"

train_patches = train_dataset.to_patches("ground_truth")

train_patches.export(
    export_dir=train_dir,
    dataset_type=fo.types.ImageClassificationDirectoryTree,
    label_field="ground_truth",
)

test_patches = test_dataset.to_patches("ground_truth")

test_patches.export(
    export_dir=test_dir,
    dataset_type=fo.types.ImageClassificationDirectoryTree,
    label_field="ground_truth",
)

if os.path.exists("/kaggle/cv/train/augmented"):
    shutil.rmtree("/kaggle/cv/train/augmented")

Directory '/kaggle/cv/train' already exists; export will be merged with existing files
Detected an image classification exporter and a label field 'ground_truth' of type <class 'fiftyone.core.labels.Detection'>. Exporting image patches...
 100% |███████████████| 1842/1842 [32.6s elapsed, 0s remaining, 56.8 samples/s]      
Directory '/kaggle/cv/test' already exists; export will be merged with existing files
Detected an image classification exporter and a label field 'ground_truth' of type <class 'fiftyone.core.labels.Detection'>. Exporting image patches...
 100% |█████████████████| 791/791 [13.2s elapsed, 0s remaining, 66.5 samples/s]      


In [207]:
!ls /kaggle/cv/train

Bird  Fish  Frog


In [208]:
!echo "Bird files:" && ls -1 /kaggle/cv/train/Bird | wc -l
!echo "Fish files:" && ls -1 /kaggle/cv/train/Fish | wc -l
!echo "Frog files:" && ls -1 /kaggle/cv/train/Frog | wc -l

Bird files:
1350
Fish files:
609
Frog files:
51


In [209]:
!ls /kaggle/cv/test

Bird  Fish  Frog


In [210]:
!echo "Bird files:" && ls -1 /kaggle/cv/test/Bird | wc -l
!echo "Fish files:" && ls -1 /kaggle/cv/test/Fish | wc -l
!echo "Frog files:" && ls -1 /kaggle/cv/test/Frog | wc -l

Bird files:
508
Fish files:
263
Frog files:
20


In [211]:
print("clearly there is class imbalance, cleaning and rebalancing is done in step4:Experiment data cleansing")

clearly there is class imbalance, cleaning and rebalancing is done in step4:Experiment data cleansing


In [212]:
print("VGG19 is a convolutional neural network model proposed by K. Simonyan and A. Zisserman from the University of Oxford in the paper -Very Deep Convolutional Networks for Large-Scale Image Recognition-. The model is 19 layers deep and was trained on the ImageNet dataset, which contains millions of images classified into 1000 categories.")

VGG19 is a convolutional neural network model proposed by K. Simonyan and A. Zisserman from the University of Oxford in the paper -Very Deep Convolutional Networks for Large-Scale Image Recognition-. The model is 19 layers deep and was trained on the ImageNet dataset, which contains millions of images classified into 1000 categories.


In [213]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
def build_vgg19_model(input_shape=(224, 224, 3), num_classes=3):
    model = Sequential([
        Input(shape=input_shape),  # Define the input shape explicitly with an Input layer
        # Block 1
        Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1'),
        Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2'),
        MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'),
        
        # Block 2
        Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1'),
        Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2'),
        MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'),
        
        # Block 3
        Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1'),
        Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2'),
        Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3'),
        Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4'),
        MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'),
        
        # Block 4
        Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1'),
        Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2'),
        Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3'),
        Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4'),
        MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'),
        
        # Block 5
        Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1'),
        Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2'),
        Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3'),
        Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4'),
        MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'),
        
        # Classification block
        Flatten(),
        Dense(4096, activation='relu', name='fc1'),
        Dropout(0.5),
        Dense(4096, activation='relu', name='fc2'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax', name='predictions')
    ])
    
    return model

# compile the model with this new definition
model = build_vgg19_model(input_shape=(224, 224, 3), num_classes=3)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [214]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/kaggle/cv/train"
test_dir = "/kaggle/cv/test"

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Size of input images expected by VGG19
    batch_size=32,
    class_mode='categorical')  # Use 'binary' if you have two classes

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',  # Use 'binary' if you have two classes
    shuffle=False)

Found 2010 images belonging to 3 classes.
Found 791 images belonging to 3 classes.


In [215]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # As per requirements
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
W0000 00:00:1711219184.130470    2245 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


11/62 ━━━━━━━━━━━━━━━━━━━━ 23s 467ms/step - accuracy: 0.4405 - loss: 1.1400

W0000 00:00:1711219232.954850    2242 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5830 - loss: 0.9042

...

62/62 ━━━━━━━━━━━━━━━━━━━━ 86s 1s/step - accuracy: 0.5840 - loss: 0.9024 - val_accuracy: 0.6615 - val_loss: 0.6741
Epoch 2/10
 1/62 ━━━━━━━━━━━━━━━━━━━━ 27s 447ms/step - accuracy: 0.7812 - loss: 0.6438

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7812 - loss: 0.6438 - val_accuracy: 0.0000e+00 - val_loss: 3.8640
Epoch 3/10


W0000 00:00:1711219261.160525    2245 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


62/62 ━━━━━━━━━━━━━━━━━━━━ 32s 513ms/step - accuracy: 0.6642 - loss: 0.7560 - val_accuracy: 0.6615 - val_loss: 0.6765
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7692 - loss: 0.6089 - val_accuracy: 0.0000e+00 - val_loss: 3.1445
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 32s 507ms/step - accuracy: 0.6622 - loss: 0.7328 - val_accuracy: 0.6615 - val_loss: 0.6787
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6562 - loss: 0.6955 - val_accuracy: 0.0000e+00 - val_loss: 3.5154
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 31s 498ms/step - accuracy: 0.6733 - loss: 0.7250 - val_accuracy: 0.6615 - val_loss: 0.6562
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7500 - loss: 0.7918 - val_accuracy: 0.0000e+00 - val_loss: 3.7627
Epoch 9/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 31s 498ms/step - accuracy: 0.6671 - loss: 0.7329 - val_accuracy: 0.6615 - val_loss: 0.6662
Epoch 10/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.5625 - loss: 0.9454 - val_accuracy: 0.0

In [216]:
# Using a pretrained model
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the base VGG19 model, without the top layer
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False


In [217]:
# Extending the base model,number of units in the Dense layer = number of classes =3
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout to reduce overfitting
predictions = Dense(3, activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)


In [218]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [219]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')


Found 2010 images belonging to 3 classes.
Found 791 images belonging to 3 classes.


In [220]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)

Epoch 1/10
 1/62 ━━━━━━━━━━━━━━━━━━━━ 6:11 6s/step - accuracy: 0.4062 - loss: 1.1398

W0000 00:00:1711219398.386616    2245 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


28/62 ━━━━━━━━━━━━━━━━━━━━ 13s 399ms/step - accuracy: 0.5438 - loss: 0.9183

W0000 00:00:1711219409.187353    2244 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step - accuracy: 0.5843 - loss: 0.8555

W0000 00:00:1711219419.888248    2242 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


62/62 ━━━━━━━━━━━━━━━━━━━━ 31s 407ms/step - accuracy: 0.5851 - loss: 0.8543 - val_accuracy: 0.6406 - val_loss: 0.6770
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7812 - loss: 0.6250 - val_accuracy: 0.6957 - val_loss: 0.5931
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 25s 366ms/step - accuracy: 0.6718 - loss: 0.7312 - val_accuracy: 0.6471 - val_loss: 0.6342
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6562 - loss: 0.6666 - val_accuracy: 0.6522 - val_loss: 0.5664
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 25s 371ms/step - accuracy: 0.6967 - loss: 0.6708 - val_accuracy: 0.6797 - val_loss: 0.5891
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7500 - loss: 0.6096 - val_accuracy: 0.6522 - val_loss: 0.6136
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 25s 369ms/step - accuracy: 0.7361 - loss: 0.6077 - val_accuracy: 0.7617 - val_loss: 0.5570
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7500 - loss: 0.5706 - val_accuracy: 0.7391 - val_los

### Experiment with Data Cleansing

Defined automatic approach to deleting blurry, under/overexposed images and noisy images

In [221]:
pip install opencv-python-headless

Note: you may need to restart the kernel to use updated packages.


In [222]:
import cv2 as cv
import numpy as np

def is_blurry(image_obj, threshold=120):
    image = np.array(image_obj)
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    variance = cv.Laplacian(gray, cv.CV_64F).var()
    return variance < threshold

def is_poorly_exposed(image_obj, low_threshold=10, high_threshold=245, ratio_threshold=0.17):
    image = np.array(image_obj)
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    hist = cv.calcHist([gray], [0], None, [256], [0, 256])
    
    low_light = np.sum(hist[:low_threshold])
    high_light = np.sum(hist[high_threshold:])
    
    total_pixels = gray.size
    return (low_light/total_pixels > ratio_threshold) or (high_light/total_pixels > ratio_threshold)


def is_noisy(image_obj, s_thr=0.15):
    image = np.array(image_obj)
    # Convert image to HSV color space
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)

    # Calculate histogram of saturation channel
    s = cv.calcHist([image], [1], None, [256], [0, 256])

    # Calculate percentage of pixels with high saturation
    p = 0.05  # Considering pixels with saturation >= p
    s_perc = np.sum(s[int(p * 255):]) / np.prod(image.shape[0:2])

    return s_perc < s_thr  # Adjusted logic and threshold

In addition duplicate images and images that are below the 32x32 minimum resolution of vgg19 will be removed

In [223]:
import os
from PIL import Image
import imagehash

def remove_bad_images(dataset_dir, min_width=10, min_height=10):
    hashes = {}
    duplicates = []
    corrupted_or_small = []
    removed_files = []
    blurry = []
    noisy = []
    poor_exposure = []

    for fileclass in os.listdir(dataset_dir):
        print(fileclass)
        category_dir = dataset_dir + "/" + fileclass
        
        for filename in os.listdir(category_dir):
            
            filepath = os.path.join(category_dir, filename)
            try:
                with Image.open(filepath) as img:
                    # Check for low resolution
                    if img.width < min_width or img.height < min_height:
                        corrupted_or_small.append(filename)
                        continue
                    
                    if is_noisy(img):
                        noisy.append(filename)
                        continue
                        
                    if is_blurry(img):
                        blurry.append(filename)
                        continue
                    
                    if is_poorly_exposed(img):
                        poor_exposure.append(filename)
                        continue
                    
    
                    # Compute hash for duplicate detection
                    hash = imagehash.average_hash(img)
                    if hash in hashes:
                        duplicates.append(filename)
                        continue
                    else:
                        hashes[hash] = filename
                        
            except cv.error as e:
                corrupted_or_small.append(filename)
                continue
        
            except (IOError, OSError):
                corrupted_or_small.append(filename)
                continue
                
        # Remove corrupted or small images
        for filename in corrupted_or_small + duplicates + poor_exposure + blurry + noisy:
            try:
                os.remove(os.path.join(category_dir, filename))
                removed_files.append(filename)
            except (IOError):
                continue
        print(f"Found  {len(noisy)} noisy images")
        print(f"Found  {len(blurry)} blurry images")
        print(f"Found  {len(poor_exposure)} bad exposure images")
        print(f"Found  {len(duplicates)} duplicate images")
        print(f"Found  {len(corrupted_or_small)} too small res images")

    return removed_files


train_removed_files = remove_bad_images(train_dir)


Fish
Found  0 noisy images
Found  143 blurry images
Found  12 bad exposure images
Found  76 duplicate images
Found  19 too small res images
Bird
Found  13 noisy images
Found  256 blurry images
Found  98 bad exposure images
Found  82 duplicate images
Found  85 too small res images
Frog
Found  13 noisy images
Found  264 blurry images
Found  105 bad exposure images
Found  82 duplicate images
Found  85 too small res images


In [224]:
!echo "Bird files:" && ls -1 /kaggle/cv/train/Bird | wc -l
!echo "Fish files:" && ls -1 /kaggle/cv/train/Fish | wc -l
!echo "Frog files:" && ls -1 /kaggle/cv/train/Frog | wc -l

Bird files:
864
Fish files:
359
Frog files:
19


In [225]:
!echo "Bird files:" && ls -1 /kaggle/cv/test/Bird | wc -l
!echo "Fish files:" && ls -1 /kaggle/cv/test/Fish | wc -l
!echo "Frog files:" && ls -1 /kaggle/cv/test/Frog | wc -l

Bird files:
508
Fish files:
263
Frog files:
20


### Class balancing via kaggle

In [226]:
from keras.preprocessing.image import load_img, img_to_array, save_img

def upsample_minority_class(minority_class_dir, augmented_class_dir, target_number):
    # Make the directory if it doesn't exist
    if os.path.exists(augmented_class_dir):
        shutil.rmtree(augmented_class_dir)
    os.makedirs(augmented_class_dir)
    print(f"The directory {augmented_class_dir} has been created")

    # List all files in the minority class directory
    minority_images = [os.path.join(minority_class_dir, f) for f in os.listdir(minority_class_dir)]

    num_new_images = target_number - len(minority_images)

    # Generate new images using data augmentation
    for i in range(num_new_images):
        img_path = np.random.choice(minority_images)
        img = load_img(img_path)
        img_array = img_to_array(img)  
        img_array = img_array.reshape((1,) + img_array.shape)
        
        for batch in augmentation_datagen.flow(img_array, batch_size=1, save_to_dir=augmented_class_dir, save_prefix='aug_', save_format='jpeg'):
            break
    
    if os.path.exists(minority_class_dir):
        shutil.rmtree(minority_class_dir)
        print(f"The directory {minority_class_dir} has been removed")
    else:
        print(f"The directory {minority_class_dir} does not exist")


In [227]:
minority_class_dir = '/kaggle/cv/train/Frog/'
augmented_class_dir = '/kaggle/cv/train/augmented/Frog/'
target_number = 350

upsample_minority_class(minority_class_dir, augmented_class_dir, target_number)

The directory /kaggle/cv/train/augmented/Frog/ has been created
The directory /kaggle/cv/train/Frog/ has been removed


In [228]:
!echo "Bird files:" && ls -1 /kaggle/cv/train/Bird | wc -l
!echo "Fish files:" && ls -1 /kaggle/cv/train/Fish | wc -l
!echo "Frog files:" && ls -1 /kaggle/cv/train/augmented/Frog | wc -l

Bird files:
864
Fish files:
359
Frog files:
329


### Perform Data Augmentation and Class balancing

In [229]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Set up data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    brightness_range=[0.4,1.5],
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    
    target_size=(224, 224),  
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224), 
    batch_size=32,
    class_mode='categorical',
    shuffle=False  
)


Found 1552 images belonging to 3 classes.
Found 791 images belonging to 3 classes.


Automatic approach to deleting blurry, under/overexposed images and noisy images

In [230]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes)

class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

In [231]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers, models
from tensorflow.keras.metrics import Precision, Recall


base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout to reduce overfitting
predictions = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])


model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=21,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size,
    class_weight=class_weights_dict  # Apply class weights
)

Epoch 1/21


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 1/48 ━━━━━━━━━━━━━━━━━━━━ 5:26 7s/step - accuracy: 0.2500 - loss: 1.4599 - precision_15: 0.2917 - recall_15: 0.2188

W0000 00:00:1711219574.999120    2242 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step - accuracy: 0.2825 - loss: 1.2019 - precision_15: 0.3048 - recall_15: 0.1176

W0000 00:00:1711219594.145853    2242 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


48/48 ━━━━━━━━━━━━━━━━━━━━ 29s 479ms/step - accuracy: 0.2838 - loss: 1.2007 - precision_15: 0.3059 - recall_15: 0.1179 - val_accuracy: 0.4479 - val_loss: 1.0518 - val_precision_15: 0.0000e+00 - val_recall_15: 0.0000e+00
Epoch 2/21
 1/48 ━━━━━━━━━━━━━━━━━━━━ 7s 162ms/step - accuracy: 0.3750 - loss: 1.0115 - precision_15: 0.2857 - recall_15: 0.0625

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3750 - loss: 1.0115 - precision_15: 0.2857 - recall_15: 0.0625 - val_accuracy: 0.9130 - val_loss: 0.7078 - val_precision_15: 1.0000 - val_recall_15: 0.6087
Epoch 3/21
48/48 ━━━━━━━━━━━━━━━━━━━━ 23s 427ms/step - accuracy: 0.4541 - loss: 1.0191 - precision_15: 0.4833 - recall_15: 0.1857 - val_accuracy: 0.5573 - val_loss: 0.9477 - val_precision_15: 0.5573 - val_recall_15: 0.1393
Epoch 4/21
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4688 - loss: 1.0663 - precision_15: 0.3333 - recall_15: 0.1875 - val_accuracy: 0.8696 - val_loss: 0.6411 - val_precision_15: 1.0000 - val_recall_15: 0.6087
Epoch 5/21
48/48 ━━━━━━━━━━━━━━━━━━━━ 23s 417ms/step - accuracy: 0.4786 - loss: 0.9590 - precision_15: 0.5422 - recall_15: 0.2833 - val_accuracy: 0.6680 - val_loss: 0.8857 - val_precision_15: 0.7182 - val_recall_15: 0.3086
Epoch 6/21
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5625 - loss: 0.8954 - precision_15: 0.6500 - recall_15: 0.40

In [232]:
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")


24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 153ms/step - accuracy: 0.6267 - loss: 0.7750 - precision_15: 0.6943 - recall_15: 0.5138
Test Loss: 0.7112874984741211
Test Accuracy: 0.6979166865348816
Test Precision: 0.7772020697593689
Test Recall: 0.5859375


In [237]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

# Callback for early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Callback for model checkpointing
model_checkpoint = ModelCheckpoint(
    'best_model.keras', monitor='val_loss', save_best_only=True, save_weights_only=False)

# Learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return float(lr * np.exp(-0.1))

lr_schedule = LearningRateScheduler(scheduler)


model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=21,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size,
    class_weight=class_weights_dict,
    callbacks=[early_stopping, model_checkpoint, lr_schedule]
)


Epoch 1/21
 1/48 ━━━━━━━━━━━━━━━━━━━━ 5:23 7s/step - accuracy: 0.6562 - loss: 0.6754 - precision_19: 0.7083 - recall_19: 0.5312

W0000 00:00:1711220195.296441    2243 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step - accuracy: 0.6704 - loss: 0.7348 - precision_19: 0.7435 - recall_19: 0.5376

W0000 00:00:1711220214.164946    2244 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


48/48 ━━━━━━━━━━━━━━━━━━━━ 30s 484ms/step - accuracy: 0.6704 - loss: 0.7350 - precision_19: 0.7436 - recall_19: 0.5378 - val_accuracy: 0.6706 - val_loss: 0.7321 - val_precision_19: 0.7543 - val_recall_19: 0.5755 - learning_rate: 1.0000e-05
Epoch 2/21
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8125 - loss: 0.6381 - precision_19: 0.9545 - recall_19: 0.6562 - val_accuracy: 0.9130 - val_loss: 0.4414 - val_precision_19: 0.9500 - val_recall_19: 0.8261 - learning_rate: 1.0000e-05
Epoch 3/21
48/48 ━━━━━━━━━━━━━━━━━━━━ 23s 426ms/step - accuracy: 0.6549 - loss: 0.7330 - precision_19: 0.7282 - recall_19: 0.5199 - val_accuracy: 0.6641 - val_loss: 0.7298 - val_precision_19: 0.7319 - val_recall_19: 0.5651 - learning_rate: 1.0000e-05
Epoch 4/21
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5312 - loss: 0.7157 - precision_19: 0.6154 - recall_19: 0.5000 - val_accuracy: 0.9130 - val_loss: 0.4503 - val_precision_19: 0.9500 - val_recall_19: 0.8261 - learning_rate: 1.0000e-05
Epoch 5/21
4

In [238]:
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")


24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 146ms/step - accuracy: 0.5989 - loss: 0.8078 - precision_19: 0.6536 - recall_19: 0.4934
Test Loss: 0.7326685786247253
Test Accuracy: 0.66796875
Test Precision: 0.7465986609458923
Test Recall: 0.5716145634651184
